In [1]:
# Useful libs
from sklearn.datasets import fetch_20newsgroups
import pickle
import pandas as pd
import numpy as np
import spacy

# STTM lib from Github
from gsdmm import MovieGroupProcess

# Load the 20NewsGroups dataset from sklearn
cats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

In [15]:
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()
docs = [nlp(text) for text in newsgroups_train['data']]

In [16]:
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)
y = mgp.fit(docs, vocab_size=len(spacy_nlp.vocab.strings))

In stage 0: transferred 1560 clusters with 7 clusters populated
In stage 1: transferred 63 clusters with 10 clusters populated
In stage 2: transferred 24 clusters with 10 clusters populated
In stage 3: transferred 11 clusters with 10 clusters populated
In stage 4: transferred 17 clusters with 10 clusters populated
In stage 5: transferred 18 clusters with 10 clusters populated


KeyboardInterrupt: 

# Ab hier relevant

In [61]:
app = pd.read_csv('text_data/appstore_review.csv')['text'].dropna().to_list()
fb_post = pd.read_csv('text_data/fb_post.csv')['text'].dropna().to_list()
gmaps = pd.read_csv('text_data/google_maps_review.csv')['text'].dropna().to_list()
tweet = pd.read_csv('text_data/tweet.csv')['text'].dropna().to_list()
social_media_data_old = [*app, *fb_post, *gmaps, *tweet]

In [69]:
import pickle
with open("text_data.pkl", "rb") as fp:
    social_media_data = pickle.load(fp)
    
social_media_data = [val for val in social_media_data if val is not None]

In [70]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [71]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [72]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print(len(docs_raw))

11314


In [82]:
from stop_words import get_stop_words
stop_words = [
    *get_stop_words("german"),
    *get_stop_words("english"),
    *["http", "https", "www", "com", "de", "google", "translated", "twitter", "fur", "uber", "html", "barberini", "museumbarberini", "museum"]
]

In [83]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(social_media_data)
print(dtm_tf.shape)

/home/tom/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'konnen', 'konnte', 'let', 'mustn', 'shan', 'shouldn', 'wahrend', 'wasn', 'weren', 'won', 'wouldn', 'wurde', 'wurden'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(8044, 1817)


In [84]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(social_media_data)
print(dtm_tfidf.shape)

/home/tom/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)
/home/tom/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'konnen', 'konnte', 'let', 'mustn', 'shan', 'shouldn', 'wahrend', 'wasn', 'weren', 'won', 'wouldn', 'wurde', 'wurden'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(8044, 1817)


In [85]:
# for TF DTM
lda_tf = LatentDirichletAllocation(random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## LDA based on TF

In [86]:
p = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
pyLDAvis.save_html(p, 'topics_tf.html')
p

/home/tom/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.103690 -0.014303       1        1  11.982569
5      0.056251 -0.258715       2        1  10.731633
9      0.078008 -0.215326       3        1  10.576008
3      0.056978 -0.006031       4        1  10.509033
4      0.241112  0.228216       5        1  10.439125
8      0.161664  0.095918       6        1   9.642113
0     -0.102590  0.088361       7        1   9.528293
6     -0.287652  0.063680       8        1   9.192023
1     -0.252644  0.045368       9        1   9.067180
7     -0.054819 -0.027167      10        1   8.332022, topic_info=     Category        Freq     Term       Total  loglift  logprob
1621  Default  884.000000      van  884.000000  30.0000  30.0000
1087  Default  830.000000    monet  830.000000  29.0000  29.0000
683   Default  745.000000     gogh  745.000000  28.0000  28.0000
1248  Default  556.000000  picasso  556.000000  27.0000  27.0000
59    Default  642.000000      app  642.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
1403  Topic10   37.835135    sehen  278.325716   0.4895  -5.0773
1261  Topic10   34.794757      pnn  251.708605   0.5063  -5.1611
915   Topic10   35.134488   kultur  380.413456   0.1030  -5.1514
773   Topic10   33.697736    immer  241.619309   0.5151  -5.1931
1728  Topic10   31.700759    werke  197.787507   0.6542  -5.2542

[597 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         7  0.224214      absolut
3         9  0.044843      absolut
3        10  0.695062      absolut
5         5  0.972215  abstraction
7         4  0.534087  abstraktion
...     ...       ...          ...
1790     10  0.239411         zeit
1794      1  0.275400      zeitung
1794      2  0.159442      zeitung
1794      6  0.550800      zeitung
1805      8  0.950236      zuhause

[1119 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 10, 4, 5, 9, 1, 7, 2, 8])

## LDA based on TFIDF

In [87]:
p = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)
pyLDAvis.save_html(p, 'topics_tfidf.html')
p

/home/tom/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.275457  0.006838       1        1  11.475982
3     -0.032382  0.022008       2        1  11.282443
4     -0.178213 -0.184741       3        1  10.695989
2     -0.133887  0.012020       4        1  10.602156
9     -0.057082  0.130097       5        1  10.205703
1      0.197868 -0.062770       6        1   9.759082
7      0.102703  0.035973       7        1   9.560437
5     -0.093878  0.206187       8        1   9.273885
8     -0.091224 -0.046980       9        1   8.682150
0      0.010640 -0.118631      10        1   8.462173, topic_info=     Category        Freq         Term       Total  loglift  logprob
1621  Default  117.000000          van  117.000000  30.0000  30.0000
683   Default  101.000000         gogh  101.000000  29.0000  29.0000
1248  Default  109.000000      picasso  109.000000  28.0000  28.0000
59    Default  141.000000          app  141.000000  27.0000  27.0000
1087  Default  129.000000        monet  129.000000  26.0000  26.0000
...       ...         ...          ...         ...      ...      ...
1247  Topic10   23.784006          pic  233.932064   0.1835  -4.2625
921   Topic10   18.888587        kunst  121.127802   0.6113  -4.4930
997   Topic10   10.919228        lohnt   26.065192   1.5995  -5.0410
110   Topic10   15.254284  ausstellung  234.862118  -0.2646  -4.7067
672   Topic10   10.371081         gibt   47.962541   0.9382  -5.0925

[582 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         2  0.049374      absolut
3         9  0.888736      absolut
5         3  0.933529  abstraction
7         2  0.367162  abstraktion
7         3  0.629421  abstraktion
...     ...       ...          ...
1794      2  0.111729      zeitung
1794      4  0.111729      zeitung
1794      8  0.167594      zeitung
1794      9  0.558647      zeitung
1805      1  0.918472      zuhause

[1218 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 3, 10, 2, 8, 6, 9, 1])

### Nächste Schritte:

- LDA Algorithmus besser verstehen
- Weitere Algorithmen ausprobieren
- Prototyp in PowerBI integrieren (Zeiträume und Datenquellen auswählbar. Vllt auch Algorithmus auswählbar)
- Feedback und Verbesserungen

### Offene Fragen:

- Wie können wir einen guten Vergleich zwischen Zeiträumen bzw. eine Betrachtung der zeitlichen Entwicklung ermöglichen?

# LDA

In [108]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from stop_words import get_stop_words
import pandas as pd
import numpy as np
import datetime as dt

    
def cluster(train, data, model_name):
    """
    Input: train Liste von Tupeln (id, text). data ist dataframe
    Output: Tupel (Liste von Tupeln (id, topic), Liste von Tupeln (topic, Liste top words))
    """
    
    training_texts = [elem[1] for elem in train]
    (lda, tf_vectorizer, dtm_tf) = get_models(training_texts)
    # (lda, tf_vectorizer, dtm_tf), (nmf, tfidf_vectorizer, dtm_tfidf) = get_models(texts)
    
    top_words_lda = get_top_words_for_topics(lda, tf_vectorizer, 30)
    # top_words_nmf = get_top_words_for_topics(nmf, tfidf_vectorizer, 30)
    
    text_topics_lda = get_topic_for_texts(lda, tf_vectorizer, data["text"].to_list())
    # text_topics_nfm = get_topic_for_texts(nmf, dtm_tfidf)
    
    # top words for topics: df with cols topic, word, position
    temp = []
    for topic, words in top_words_lda:
        for i, word in enumerate(words):
            temp.append({'topic': topic, 'term': word, 'position': i, 'model': model_name})
    topics = pd.DataFrame(temp)
    
    # topics for text: df with cols id, text, topic
    temp = pd.DataFrame([d[1] for d in text_topics_lda], columns=["topic"])
    temp.index += 1
    temp["model"] = model_name
    texts = pd.merge(text_data, temp, left_index=True, right_index=True)

    return topics, texts

def get_models(texts):
    stop_words = [
        *get_stop_words("german"),
        *get_stop_words("english"),
        *["http", "https", "www", "com", "de", "google", "translated", "twitter", "fur", "uber", "html", "barberini", "museumbarberini", "museum"]
    ]

    # -------- LDA with tf vectorizer ---------
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    stop_words = stop_words,
                                    lowercase = True,
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    max_df = 0.5, 
                                    min_df = 10,
                                    max_features=2000)
    dtm = tf_vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(
        n_components=10, max_iter=5, learning_method='online', 
        learning_offset=50.,random_state=0
    ).fit(dtm)
    
    return lda, tf_vectorizer, dtm

    # -------- NMF with tfidf vectorizer -----------
    # tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
    # tfidf = tfidf_vectorizer.fit_transform(texts)
    # nmf = NMF(n_components=7, random_state=1, 
    #     alpha=.1, l1_ratio=.5, init='nndsvd'
    # ).fit(dtm)
    # 
    # return (lda, tf_vectorizer, dtm), (nmf, tfidf_vectorizer, tfidf)

def get_top_words_for_topics(model, vectorizer, no_top_words):
    feature_names = vectorizer.get_feature_names()
    out = []
    for topic_idx, topic in enumerate(model.components_):
        out.append(
            (topic_idx, [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        )
    return out

def get_topic_for_texts(model, vectorizer, texts):
    dtm = vectorizer.transform(texts)
    out = []
    doc_topic = model.transform(dtm)
    for n in range(doc_topic.shape[0]):
        topic_most_pr = doc_topic[n].argmax()
        out.append((n, topic_most_pr))
    return out

In [109]:
text_data = pd.read_csv("text_data.csv").dropna()
text_data["post_date"] = pd.to_datetime(text_data["post_date"])

In [110]:
# train topic model on all the data
texts = [(i, t) for i, t in enumerate(text_data["text"].to_list())]
topics, texts = cluster(texts, text_data, "full")

/home/tom/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'konnen', 'konnte', 'let', 'mustn', 'shan', 'shouldn', 'wahrend', 'wasn', 'weren', 'won', 'wouldn', 'wurde', 'wurden'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [111]:
# train topic model on data from 2019
data_2019 = text_data[text_data['post_date'].dt.year == 2019]
texts_2019 = [(i,t) for i,t in enumerate(data_2019["text"].to_list())]
topics_2019, texts_2019 = cluster(texts_2019, text_data, "2019")

/home/tom/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'konnen', 'konnte', 'let', 'mustn', 'shan', 'shouldn', 'wahrend', 'wasn', 'weren', 'won', 'wouldn', 'wurde', 'wurden'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [112]:
pd.concat([topics, topics_2019]).to_csv("topics_and_terms.csv")
pd.concat([texts, texts_2019]).to_csv("topics_and_texts.csv")

# GSDMM

### Resources

- The GSDMM paper: http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf
- A blog post about STTM: https://towardsdatascience.com/short-text-topic-modeling-70e50a57c883
- The gsdmm python package: https://github.com/rwalk/gsdmm

In [1]:
from stop_words import get_stop_words
import pickle
import pandas as pd
import numpy as np
from gsdmm import MovieGroupProcess
from nltk.tokenize import word_tokenize, MWETokenizer
import nltk
from collections import defaultdict
import langdetect
from pprint import pprint

class Doc:
    def __init__(self, text, source, post_date):
        self.text = text
        self.source = source
        self.post_date = post_date
        self.tokens = None
        self.topic = None
        self.model_name = None
        self.language = None
    
    def in_year(self, year):
        if year == "all":
            return True
        return year in self.post_date
        
    def too_short(self):
        return len(self.tokens) <= 2
    
    def predict(self, model, model_name):
        self.topic = model.choose_best_label(self.tokens)[0]
        self.model_name = model_name
        
    def to_dict(self):
        return {
            "text": self.text,
            "source": self.source,
            "post_date": self.post_date,
            "topic": self.topic,
            "model_name": self.model_name
        }

In [2]:
def preprocess(docs):
    """
    docs: list of Doc objects
    
    returns: list of relevant Doc objects, pre-processed
    """
    
    # remove leading "None" (introduced by DB export)
    for doc in docs:
        doc.text = doc.text.replace("None ", "", 1)
    
    # lowercasing
    for doc in docs:
        doc.tokens = doc.text.lower()

    # detect language
    for doc in docs:
        try:
            doc.language = langdetect.detect(doc.text)
        except:
            print(doc.to_dict())

    # consider only german docs
    docs = [doc for doc in docs if doc.language == "de"]
            
    # tokenize
    for doc in docs:
        doc.tokens = word_tokenize(doc.tokens)
        
    # re-tokenize so that some names are not broken into multiple tokens
    tokenizer = MWETokenizer([
        ("van", "gogh"), ("museum", "barberini"), ("museum", "barberini", "potsdam"),
        ("max", "beckmann"), ("wege", "des", "barock")
    ])
    for doc in docs:
        doc.tokens = tokenizer.tokenize(doc.tokens)
    
    # remove stop words
    stop_words = [
            *get_stop_words("german"),
            *get_stop_words("english"),
            *["http", "https", "www", "com", "de", "google", "translated", 
              "twitter", "fur", "uber", "html", "barberini", 
              "museumbarberini", "museum", "ausstellung", "ausstellungen",
              "potsdam"]
    ]
    for doc in docs:
        doc.tokens = [token for token in doc.tokens if token not in stop_words]

    # stemming
    #stemmer = nltk.stem.cistem.Cistem()
    #for doc in docs:
    #    doc.tokens = [stemmer.stem(token) for token in doc.tokens]
        
    # keep only alphabetical tokens
    for doc in docs:
        doc.tokens = [token for token in doc.tokens if token.isalpha()]
        
    # remove single-digit tokens
    for doc in docs:
        doc.tokens = [token for token in doc.tokens if len(token) > 1]
    
    # remove tokens that appear only once
    tokens = defaultdict(lambda: 0)
    for doc in docs:
        for token in doc.tokens:
            tokens[token] += 1
    for doc in docs:
        for token in doc.tokens:
            if tokens[token] == 1:
                doc.tokens.remove(token)
                
    # remove docs with less than three tokens
    docs = [doc for doc in docs if not doc.too_short()]
            
    return docs

def print_stats(docs):
    print(f"number of documents: {len(docs)}")
    vocab = set(x for doc in docs for x in doc.tokens)
    n_terms = len(vocab)
    print(f"vocabulary size: {n_terms}")
    
def train_mgp(docs, K=50, alpha=0.1, beta=0.1, n_iters=10):
    mgp = MovieGroupProcess(K=K, alpha=alpha, beta=beta, n_iters=n_iters)

    vocab = set(x for doc in docs for x in doc.tokens)
    n_terms = len(vocab)
    
    y = mgp.fit([doc.tokens for doc in docs], n_terms)

    # Save model
    with open("topic_model.pkl", "wb") as f:
        pickle.dump(mgp, f)
        f.close()
        
    return mgp

def top_terms(model, n=20):
    
    return [
        sorted(list(doc_dist.items()), key = lambda x: x[1], reverse=True)[:n]
        for doc_dist in model.cluster_word_distribution
    ]

def get_dfs(model, docs, model_name="all"):
    # text,source,post_date,topic,model
    for doc in docs:
        doc.predict(model, model_name)
    text_df = pd.DataFrame([doc.to_dict() for doc in docs])

    # topic,term,count,model
    out = []
    for i, topic_terms in enumerate(top_terms(model)):
        for term in topic_terms:
            out.append({
                "topic": i,
                "term": term[0],
                "count": term[1],
                "model": model_name
            })
    topic_df = pd.DataFrame(out)
    
    # topic,model,title
    out = []
    top = top_terms(model)
    for i in range(len(top)):
        out.append({
            "model": model_name,
            "topic": i,
            "title": max(top[i], key=lambda x: x[1])[0] if top[i] else None
        })
    title_df = pd.DataFrame(out)
    
    return topic_df, text_df, title_df
    

In [3]:
# read data
print("reading data..")
data = pd.read_csv("../text_data.csv").dropna()

# exclude facebook posts
data = data[data["source"] != "facebook"]

# create list of Docs
print("creating list of Docs..")
docs = []
for row in data.itertuples():
    docs.append(Doc(row.text, row.source, row.post_date))
    
# preprocessing
print("preprocessing..")
relevant_docs = preprocess(docs)
print_stats(relevant_docs)

reading data..
creating list of Docs..
preprocessing..
{'text': '👍', 'source': 'twitter', 'post_date': '2018-05-15 09:09:44', 'topic': None, 'model_name': None}
{'text': '♥️', 'source': 'google_maps', 'post_date': '2019-06-01 00:00:00', 'topic': None, 'model_name': None}
{'text': 'https://jachwe.com/2019/11/24/rauchverordnung-am-rande-des-neuen-biedermeiers/\xa0…', 'source': 'twitter', 'post_date': '2019-11-24 07:06:13', 'topic': None, 'model_name': None}
{'text': '🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞🐞😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂\nGute Nacht Ihr Lieben!', 'source': 'twitter', 'post_date': '2017-01-27 21:47:03', 'topic': None, 'model_name': None}
{'text': '😊', 'source': 'twitter', 'post_date': '2018-05-04 13:48:22', 'topic': None, 'model_name': None}
{'text': '♡', 'source': 'google_maps', 'post_date': '2019-12-05 00:00:00', 'topic': None, 'model_name': None}
{'text': 'http://m.maz-online.de/Nachrichten/Kultur/Kurator-Dietmar-Elger-zur-Gerhard-Richter-Ausstellung-im-Museum-Barberini-in-Potsdam\xa0…', 'source': 'twi

In [5]:
# one model per year
topics, texts, titles = [], [], []
for model_name in ["all", "2020", "2019", "2018", "2017", "2016"]:
    print(f"training model '{model_name}'")
    
    data_in_timespan = [doc for doc in relevant_docs if doc.in_year(model_name)]
    if model_name == "all":
        model = train_mgp(data_in_timespan, K=15, n_iters=15)
    else:
        model = train_mgp(data_in_timespan, K=10, n_iters=15)
    topic_df, text_df, title_df = get_dfs(model, data_in_timespan, model_name)
    
    topics.append(topic_df)
    texts.append(text_df)
    titles.append(title_df)
    
print("writing dfs to disk..")
pd.concat(texts).to_csv("text_df_time_constraint.csv")
pd.concat(topics).to_csv("topic_df_time_constraint.csv")
pd.concat(titles).to_csv("title_df_time_constraint.csv")

training model 'all'
In stage 0: transferred 3923 clusters with 15 clusters populated
In stage 1: transferred 2435 clusters with 15 clusters populated
In stage 2: transferred 1715 clusters with 15 clusters populated
In stage 3: transferred 1406 clusters with 15 clusters populated
In stage 4: transferred 1280 clusters with 15 clusters populated
In stage 5: transferred 1184 clusters with 15 clusters populated
In stage 6: transferred 1068 clusters with 15 clusters populated
In stage 7: transferred 1055 clusters with 15 clusters populated
In stage 8: transferred 968 clusters with 15 clusters populated
In stage 9: transferred 952 clusters with 15 clusters populated
In stage 10: transferred 880 clusters with 15 clusters populated
In stage 11: transferred 884 clusters with 15 clusters populated
In stage 12: transferred 891 clusters with 15 clusters populated
In stage 13: transferred 882 clusters with 15 clusters populated
In stage 14: transferred 823 clusters with 15 clusters populated
traini

In [5]:
# ideen:
# - modell pro datenquelle
# - facebook rausnehmen (weil texte auf facebook eher lang sind)